# Custom templates

## Colab-specific setup

Make sure you have a Database, a Secure Connect Bundle and a Database Token. See [DB setup](https://cassio.org/db_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.
Get ready to upload the Bundle and supply the Token string.

Likewise, ensure you have the necessary secret for a LLM provider: you'll be asked to input it shortly. See [Api setup](https://cassio.org/api_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.

_Note: this colab is autogenerated from a regular Jupyter notebook hosted by cassio.org. Run all cells in this section to complete the setup before moving on to the demo content proper._

In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio#egg=langchain" \
    "git+https://github.com/datastax/python-driver.git@cep-vsearch#egg=cassandra-driver" \
    "cassio>=0.0.2" \
    "feast>=0.26" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_APPLICATION_TOKEN = input('Your Astra DB Token: ')

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# Set your secret(s) for LLM access (key names must match `providerValidator` in `llm_choice`)
llmProvider = 'OpenAI'  # 'VertexAI'
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
    ExecutionProfile,
    EXEC_PROFILE_DEFAULT,
    ConsistencyLevel,
)
from cassandra.auth import PlainTextAuthProvider

ASTRA_DB_CLIENT_ID = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        profile = ExecutionProfile(
            consistency_level=ConsistencyLevel.LOCAL_QUORUM,
        )
        #
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_APPLICATION_TOKEN,
            ),
            execution_profiles={EXEC_PROFILE_DEFAULT: profile},
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unknown CQL Session mode')

# Custom templates


Prompt templates that automatically bind to a database are built on top of a specialized kind of prompt template in LangChain called `DependencyfulPromptTemplate`. This kind of prompt has two abilities:

1. it retain "dependencies" (such as a database connection) throughout its lifetime;
2. it admits named arguments to the `format` method that will be routed to its own "getter" function, instead of directly replacing the placeholder variables in the prompt string.

The Cassandra-backed prompt template then builds a certain `DependencyfulPromptTemplate` bound to the Cassandra database in the right way. Thanks to metadata inspection, detailed knowledge about the DB schema is not provided explicitly when creating the prompt, as was shown earlier.

As another example, the Feast template (shown in a later page) builds a different `DependencyfulPromptTemplate` which "knows" how to connect to Feast and retrieve the needed information to later fill the template string.

### Customization

However, sometimes one may need to go beyond the ready-made templates and build a custom prompt. This page shows how this is done.

Let's build a mock "account balance checker" that associates a current balance to each user (a key-value store), but it also keeps a counter of how many times a certain balance is accessed.

In [1]:
class Balancer():
    def __init__(self, accounts):
        self.accounts = accounts
        self._accesses = {}
        
    def read(self, user):
        if user in self.accounts:
            self._accesses[user] = self._accesses.get(user, 0) + 1
        return self.accounts[user]
    
    def accesses(self):
        return self._accesses

In [2]:
bal0 = Balancer({'Rita': 100, 'Otto': 40, 'Anna': 150})
bal0.read('Rita')
bal0.read('Rita')
bal0.read('Anna')
print(bal0.accesses())

{'Rita': 2, 'Anna': 1}


Ok, now let's create another balancer for use in the template:

In [3]:
myBalance = Balancer({'Katie': 1000, 'Gil': 800, 'Hugh': 1200})

### Template creation

#### The template string

In [4]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [5]:
balancePrompt = ("Hello. Your balance is {balance}. "
                 "You have {unreads} unread message(s).")

#### Dependencies

This template will have a single dependency, the `Balancer` instance itself:

In [6]:
dependencyDict = {'balancer': myBalance}

#### Getter

We need a "getter" function, which will be called in the `format` method of the template and whose first argument is the dict of dependencies (in this case the `Balancer` instance), followed by any other keyword argument.

The getter must return a dict with values for all variables in the above `PromptTemplate` that will be retrieved from the storage dependency, i.e. not supplied directly at `format` time:

In [7]:
def balanceGetter(deps, **kwargs):
    userName = kwargs['user_name']
    balancer = deps['balancer']
    return {'balance': balancer.read(userName)}

In [8]:
from langchain.prompts import DependencyfulPromptTemplate

myCustomTemplate = DependencyfulPromptTemplate(
    template=balancePrompt,
    dependencies=dependencyDict,
    getter=balanceGetter,
    input_variables=["user", "user_name"],
    forceGetterArguments=["user_name"],
)

#### Rendering

In [9]:
myCustomTemplate.format(user_name='Gil', unreads=123)

'Hello. Your balance is 800. You have 123 unread message(s).'

In [10]:
myCustomTemplate.format(user_name='Gil', unreads=456)

'Hello. Your balance is 800. You have 456 unread message(s).'

In [11]:
myCustomTemplate.format(user_name='Katie', unreads=789)

'Hello. Your balance is 1000. You have 789 unread message(s).'

In [12]:
myBalance.accesses()

{'Gil': 2, 'Katie': 1}